In [11]:
import torch
import numpy as np
import load_data
import pandas as pd
import torchtext

In [20]:
torch.cuda.set_device(3)

TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_data.load_dataset()


Length of Text Vocabulary: 44003
Vector size of Text Vocabulary:  torch.Size([44003, 300])
Label Length: 5
Label Mapping:  defaultdict(<function _default_unk_index at 0x7facf8a220d0>, {'5': 0, '4': 1, '3': 2, '2': 3, '1': 4})
Most frequent:  [('hotel', 38634), ('room', 27387), ('not', 23896), ('great', 16352), ("n't", 14391), ('good', 13047), ('staff', 12885), ('stay', 12167), ('did', 10914), ('nice', 9708), ('just', 9464), ('rooms', 9404), ('location', 9136), ('no', 8961), ('stayed', 8161), ('night', 7804), ('service', 7768), ('time', 7602), ('breakfast', 7582), ('clean', 7454)]


In [21]:
model = torch.load('rcnn_final.pth')

In [22]:
test = pd.read_csv('all/test.csv')

In [23]:
print(TEXT.preprocess(test['text'][1]))

['nice', 'hotel', 'far', 'city', 'center', 'stayed', 'hotel', 'days', 'combined', 'business', 'sightseeing', 'trip', 'attending', 'conference', 'held', 'hotel', 'probably', 'not', 'booked', 'far', 'city', 'center', 'however', 'hotel', 'excellent', 'choice', 'property', 'beautiful', 'amenities', 'expected', 'good', 'five', 'star', 'hotel', 'pool', 'spa', 'shopping', 'gym', 'tennis', 'bowling', 'excellent', 'restaurants', 'on', 'site', 'good', 'not', 'area', 'restaurants', 'generally', 'pricey', 'beijing', 'standards', 'cheaper', 'used', 'did', "n't", 'mind', 'rooms', 'spacious', 'immaculately', 'clean', 'well', 'decorated', 'bathrooms', 'large', 'lovely', 'furnishings', 'free', 'bottled', 'water', 'provided', 'rooms', 'tap', 'water', "n't", 'safe', 'drink', 'in', 'room', 'internet', 'access', 'available', 'cost', 'pay', 'logged', 'online', 'frequently', 'total', 'cost', 'stay', 'negligible', 'we', 'paid', 'reasonable', '650', 'yuan', 'night', 'including', 'buffet', 'breakfast', 'extensi

In [24]:
import torch.nn.functional as F
from torch.autograd import Variable
pred = []
for i in range(len(test['text'])):
    test_sen = TEXT.preprocess(test['text'][i])
    test_sen = np.asarray([[TEXT.vocab.stoi[x] for x in test_sen]])
    test_sen = torch.LongTensor(test_sen)
    test_tensor = Variable(test_sen, volatile=True)
    test_tensor = test_tensor.cuda()
    model.eval()
    output = model(test_tensor, 1)
    out = F.softmax(output, 1)
    out = torch.argmax(out[0]).item()
    
    if out == 0:
        out = 5
    elif out == 1:
        out = 4
    elif out == 2:
        out = 3
    elif out == 3:
        out = 2
    elif out == 4:
        out = 1
    pred.append(out)

/home/data/urop2018/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [25]:
log = pd.read_csv('rcnn_final_2.csv')
print(np.mean(pred == log['pred']))

0.8648407926965042


In [26]:
sub_df = pd.DataFrame()
sub_df["id"] = test['id']
sub_df["pred"] = pred
sub_df.to_csv("rcnn_final_3.csv", index=False)

In [27]:
att = pd.read_csv("rcnn_final_3.csv")
att

,id,pred
0,0,5
1,1,4
2,2,5
3,3,5
4,4,5
5,5,1
6,6,5
7,7,1
8,8,4
9,9,4
